# Function Calling with OpenAI & AIConfig
This notebook serves as a practical guide for leveraging AIConfig and function calling with OpenAI models. We start with a mock database of books and functions to list, search, and retrieve books. Function calling is enabled so the LLM can interpret a user's question, determine the appropriate function to call, and execute the function. Read more about [Function Calling with Open AI](https://openai.com/blog/function-calling-and-other-api-updates) and [AIConfig for prompt and model management](https://github.com/lastmile-ai/aiconfig).

In [ ]:
# Install AIConfig package
!pip3 install python-aiconfig

# Create .env file at aiconfig/.env containing the following line: 
# OPENAI_API_KEY=<your key here>
# You can get your key from https://platform.openai.com/api-keys
import openai
import dotenv
import os
dotenv.load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

## Set up the books DB

In [ ]:
# Define db of books
db = [
    {
        'id': 'a1',
        'name': 'To Kill a Mockingbird',
        'genre': 'historical',
        'description': ('Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.'),
    },
    {
        'id': 'a2',
        'name': 'All the Light We Cannot See',
        'genre': 'historical',
        'description': ('In a mining town in Germany, Werner Pfennig, an orphan, grows up with his younger sister, enchanted by a crude radio they find that brings them news and stories from places they have never seen or imagined. Werner becomes an expert at building and fixing these crucial new instruments and is enlisted to use his talent to track down the resistance. Deftly interweaving the lives of Marie-Laure and Werner, Doerr illuminates the ways, against all odds, people try to be good to one another.'),
    },
    {
        'id': 'a3',
        'name': 'Where the Crawdads Sing',
        'genre': 'historical',
        'description': ('For years, rumors of the “Marsh Girl” haunted Barkley Cove, a quiet fishing village. Kya Clark is barefoot and wild; unfit for polite society. So in late 1969, when the popular Chase Andrews is found dead, locals immediately suspect her.\n\n'
                        'But Kya is not what they say. A born naturalist with just one day of school, she takes life\'s lessons from the land, learning the real ways of the world from the dishonest signals of fireflies. But while she has the skills to live in solitude forever, the time comes when she yearns to be touched and loved. Drawn to two young men from town, who are each intrigued by her wild beauty, Kya opens herself to a new and startling world—until the unthinkable happens.'),
    },
]

## Define functions (to interact with DB)

In [ ]:
# Define the functions: list, search, get

# The 'list' function returns a list of books in a specified genre.
def list(genre):
    return [item for item in db if item['genre'] == genre]

# The 'search' function returns a list of books that match the provided name.
def search(name):
    return [item for item in db if name in item['name']]

# The 'get' function returns detailed information about a book based on its ID.
# Note: This function accepts only IDs, not names. Use the 'search' function to find a book's ID.
def get(id):
    for item in db:
        if item['id'] == id:
            return item
    return None

## Create your AIConfig

In [ ]:
from aiconfig import AIConfigRuntime, InferenceOptions

# Load the aiconfig.
config = AIConfigRuntime.create(name="Book Finder", description="Use OpenAI function calling to help recommend books")

In [ ]:

model = "gpt-3.5-turbo"
data = {
    "model": model,
    "messages": [
        {
            "role": "system",
            "content": "Please use our book database, which you can access using functions to answer the following questions."
        },
        {
            "role": "user",
            "content": "I really enjoyed reading {{book}}, could you recommend me a book that is similar and tell me why?"
        }
    ],
    "functions": [
        {
          "name": "list",
          "description": "list queries books by genre, and returns a list of names of books",
          "parameters": {
            "type": "object",
            "properties": {
              "genre": {
                "type": "string",
                "enum": [
                  "mystery",
                  "nonfiction",
                  "memoir",
                  "romance",
                  "historical"
                ]
              }
            }
          }
        },
        {
          "name": "search",
          "description": "search queries books by their name and returns a list of book names and their ids",
          "parameters": {
            "type": "object",
            "properties": {
              "name": {
                "type": "string"
              }
            }
          }
        },
        {
          "name": "get",
          "description": "get returns a book's detailed information based on the id of the book. Note that this does not accept names, and only IDs, which you can get by using search.",
          "parameters": {
            "type": "object",
            "properties": {
              "id": {
                "type": "string"
              }
            }
          }
        }
    ]
}

new_prompts = await config.serialize(model, data, prompt_name="recommend_book", params={"book": "To Kill a Mockingbird"})
config.add_prompt(prompt_name="recommend_book", prompt_data=new_prompts[0])

In [ ]:
config.save("recommender.aiconfig.json")

## Get recommendations using function calls

In [ ]:
# Use helper function to executes the function specified by the LLM's output for 'function_call'.
# It handles 'list', 'search', or 'get' functions, and raises a ValueError for unknown functions.
import json

def call_function(function_call):
    args = json.loads(function_call.arguments)
    name = function_call.name

    if name == 'list':
        return list(args['genre'])
    elif name == 'search':
        return search(args['name'])
    elif name == 'get':
        return get(args['id'])
    else:
        raise ValueError('No function found')

In [ ]:
# Run recommendBook prompt with gpt-3.5 and determine right function to call based on user question
params = {"book":"Where the Crawdads Sing"}
inference_options = InferenceOptions(stream=True)

result = await config.run("recommend_book", params, options=inference_options)
tool_calls = result[0].data.value
func_call = tool_calls[0].function


In [ ]:
# Run call_function to execute the LLM-specified function (list, search, get)
value = call_function(func_call)
print(json.dumps(value, indent=4)
)

## Using GPT to generate a user-friendly response

In [ ]:
data = {
    "model": model,
    "messages": [
        {
            "role": "user",
            "content": "Here is some data about a book from a books DB - please write a short description about the book as if you're a librarian. Data: {{book_info}}"
        }
    ]
}

new_prompts = await config.serialize(model, data, prompt_name="gen_summary")
config.add_prompt(prompt_name="gen_summary", prompt_data=new_prompts[0])

In [ ]:
result = await config.run("gen_summary", params={"book_info": value}, options=inference_options)

In [ ]:
config.save("recommender.aiconfig.json")